# Stock-Sales Strategy for a Simplified Market
This example finds a stock-selling strategy for a simplified market model to demonstrate using a Leap hybrid CQM solver on a constrained problem with integer and binary variables.

In this very simple market, you have some number of shares that you want to sell in daily blocks over a defined interval of days. <br>
Each sale of shares affects the market price of the stock, <br>
$ p_{i} = p_{i - 1} + \alpha s_{i - 1} $ <br>
where $ p_{i}$ and $s_{i}$ are, respectively, the price and the number of shares sold on day $i$, and $\alpha$ is some multiplier.
The goal of this problem is to find the optimal number of shares to sell per day to maximize revenue from the total sales.

The Market with Taxation section adds a tax to the market model to demonstrate the incorporation of binary variables into the CQM.

In [2]:
import dimod

## Formulate the Problem
First, define the market parameters:
- `max_days` is the period over which you should sell all your shares
- `total_shares` is the nuymber of shares you own (equal to $\sum_{i}s_{i}$)
- `price_day_0` is the stock price on the first day of the period
- `alpha` is a multiplier, $\alpha$, that controls how much the stock price increases for each share sold into the market

In [3]:
max_days = 10
total_shares = 100
price_day_0 = 50
alpha = 1

## Instantiate the CQM

In [4]:
from dimod import ConstrainedQuadraticModel
cqm = ConstrainedQuadraticModel()

## Formulating the Objective Function
The objective function to maximize is the revenue from selling shares. Because you own an integer number of shares, it is convenient to use integer variables to indicate the number of shares sold each day, `shares`. For simplicity, this model assumes stock prices, `price`, are also integers.

Bounds on the range of values for integer variables shrink the solution space the solver must search, so it is helpful to set such bounds; for many problems, you can find bounds from your knowledge of the problem. In this case,

- On any day, you cannot sell more than the total number of shares you start with.
- The maximum share price is the sum of the initial price and the total price increase that would result from selling all your shares. <br>
$ \max(p) = p_{0} + \alpha * \sum_{i}s_{i} $

In [5]:
from dimod import Integer
max_p = price_day_0 + alpha*total_shares
shares = [Integer(f'(s_{i}', upper_bound = total_shares) for i in range(max_days)]
price = [Integer(f'p_{i}', upper_bound = max_p) for i in range(max_days)]

Daily revenue is the number of shares sold multiplied by the price on each sales day

In [6]:
revenue = [s*p for s, p in zip(shares, price)]


To maximise total revenue, $\sum_{i} s_{i}p_{i}$, is to minimize the negative of that same revenue:

In [7]:
cqm.set_objective(-sum(revenue))

In [8]:
revenue[0]

QuadraticModel({'(s_0': 0.0, 'p_0': 0.0}, {('p_0', '(s_0'): 1.0}, 0.0, {'(s_0': 'INTEGER', 'p_0': 'INTEGER'}, dtype='float64')

### Constraints

The simplified market has the following constraints: 
1. In total, you can sell only the number of shares you own, no more, $\sum_{i} s_{i} < total\_shares$

In [9]:
cqm.add_constraint(sum(shares) <= total_shares, label = 'Sell only sahres you own')


'Sell only sahres you own'

2. on the first day of the selling period, the stock has aparticular price $p_{0} = price_day_0$


In [10]:
cqm.add_constraint(price[0] == price_day_0, label = 'Initial Share Price')


'Initial Share Price'

3. The stock price increases in proportion to the number of shares sold the previous day:
$p_{i} = p_{i - 1} + \alpha s_{i - 1}$

In [11]:
for i in range(1, max_days):
    pricing = cqm.add_constraint(price[i] - price[i - 1] - alpha*shares[i - 1] == 0, label = f'Sell at the price on day {i}')

For a sales period of ten days, this CQM has altogether 10 constrtaints

In [12]:
len(cqm.constraints)

11

## Solve the problem by sampling

In [13]:
from dwave.system import LeapHybridCQMSampler
sampler = LeapHybridCQMSampler()

In [14]:
sampler.solver.name

'hybrid_constrained_quadratic_model_version1p'

maximum allowed time is set for one minute

In [15]:
sampleset = sampler.sample_cqm(cqm, 
                                time_limit = 60, 
                                label = "Sam Stock Sales Strategy")
print("{} feasible solutions of {}.".format(
    sampleset.record.is_feasible.sum(), len(sampleset)))

108 feasible solutions of 132.


The function below extracts from the returned sampleset te best feasible solution and parses it. 

In [16]:
def parse_best(sampleset):
    best = sampleset.filter(lambda row: row.is_feasible).first
    s = [val for key, val in best.sample.items() if "s_" in key]
    p = [val for key, val in best.sample.items() if "p_" in key]
    r = [p*s for p, s in zip(p, s)]
    return  r, s, best

Parse and print the best feasible solution

In [17]:
r, s, _ = parse_best(sampleset)
print("Revenue of {} found for daily sales of: \n{}".format(sum(r), s))

Revenue of 9499.0 found for daily sales of: 
[10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 11.0, 9.0, 10.0]


## Market with Taxation
Quadratic models can also accept binary variables. In the formulatio below, the market is taxed on early sals and uses a binmary variable to incorporate that update into the CQM created in the previosu sections. 
Consider a market in which you pat a tax in amount, `tax_payment`, fopr selling shares dfuring the first `taxed_period` days of the period in which you can sell your shares. 

In [21]:
taxed_period = 3
tax_payment = 225

Because you either pay tax or you don't,it can use binary variable `t`, to indicate payment. The previous objective can be updated by reducing the revenue form the share sales by the tax payment amount if the `t` binary variable is 1:

In [22]:
from dimod import Binary
t = Binary('t')
cqm.set_objective(tax_payment*t - sum(revenue))

Binary variable, `t`, should be True (1) if sales in the first `taxed_period` days of the period are greater than zero; otherwise it should be False(0): <br>
$\sum_{i<taxed\_period} s_{i} > 0 \rightarrow t = 1 $ <br>
$\sum_{i<taxed\_period} s_{i} = 0 \rightarrow t = 0 $ <br>


One way to set such an indicator variable is to create a pair of linear constraints: <br>
$ \frac{\sum_{i < taxed\_period} s_{i}}{\sum_{i} s_{i}} \leq t \leq \sum_{i < taxed\_period} s_{i} $

Adding these two constraints to the previously created CQM

In [25]:
cqm.add_constraint(t - sum(shares[:taxed_period]) <= 0, label = "Tax Part 1")

'Tax Part 1'

In [26]:
cqm.add_constraint(1 / total_shares*sum(shares[:taxed_period]) - t <= 0, label = "Tax Part 2")


'Tax Part 2'

Submit the CQM to the selected solver. with a maximum allowed runtime of one minute

In [27]:
sampleset = sampler.sample_cqm(cqm, 
                                time_limit = 60, 
                                label = "Sam Stock Sales Strategy Trial")
print("{} feasible solutions of {}.".format(
    sampleset.record.is_feasible.sum(), len(sampleset)))

123 feasible solutions of 147.


parse and print the best feasible solution:

In [28]:
r, s, best= parse_best(sampleset)
income = sum(r) - best.sample['t']*tax_payment
print("Post-tax income of {} found for daily sales of: \n{}".format(income, s))

Post-tax income of 9282.0 found for daily sales of: 
[0.0, 0.0, 0.0, 13.0, 14.0, 15.0, 15.0, 16.0, 13.0, 14.0]


Notice that the existence of this tax, though avoided in the sales strateg found above, has reduced your income by a little less than the tax free (the maximum inciome if you had paid the tax would be 9275). If the tax is slightly reduced, it is more profitale to sell during the axation period and pay the tax. 

In [29]:
tax_payment = 220
cqm.set_objective(tax_payment*t - sum(revenue))
sampleset = sampler.sample_cqm(cqm, 
                                time_limit = 60, 
                                label = "Sam Sales Strategy Trial")
r, s, best = parse_best(sampleset)
income = sum(r) - best.sample['t']*tax_payment
print("Post-tax income of {} found for daily sales of: \n{}".format(income, s))

Post-tax income of 9283.0 found for daily sales of: 
[0.0, 0.0, 0.0, 14.0, 13.0, 14.0, 16.0, 15.0, 14.0, 14.0]
